In [1]:
import glob
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial.distance import pdist, squareform

https://programminghistorian.org/en/lessons/common-similarity-measures
##should be the reference for the all process

https://programminghistorian.org/en/lessons/analyzing-documents-with-tfidf
Analyzing Documents with TF-IDF
主要衡量词频和某类型词语出现次数与分布程度

https://programminghistorian.org/en/lessons/clustering-visualizing-word-embeddings
Clustering and Visualising Documents using Word Embeddings
词语嵌入（这个也许有用
https://github.com/programminghistorian/jekyll/blob/gh-pages/assets/clustering-visualizing-word-embeddings/clustering-visualizing-word-embeddings.ipynb

In [ ]:
%load_ext autoreload
%autoreload 1

Generally useful libraries.

In [ ]:
import pandas as pd
import numpy as np
import pickle
import math
import os
import re

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import cm

import seaborn as sns

Needed for the dimensionality reduction stage.

In [ ]:
try:
    import umap
except ModuleNotFoundError:
    print("Module not found, will try to install...")
    !pip install umap-learn
    import umap

Needed for hierarchical clustering stage.

In [ ]:
try:
    from kneed import KneeLocator
except ModuleNotFoundError:
    print("Module not found, will try to install...")
    !pip install kneed
    from kneed import KneeLocator

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster, centroid
from tabulate import tabulate

Needed for the validation and visualisation stage.

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, silhouette_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from wordcloud import WordCloud

In [ ]:
url      = 'https://github.com/MaartenGr/cTFIDF/archive/refs/tags/v0.1.1.tar.gz'
version  = re.search(r'/v(.+?)\.tar\.gz', url).group(1)
dir_name = f'cTFIDF-{version}'
dir_path = os.path.join(os.getcwd(),dir_name)

if not os.path.exists(dir_name):

    print("Module not found, will try to download and prepare...")

    import requests, tarfile

    r = requests.get(url, allow_redirects=True)
    open(f'{dir_name}.tar.gz', 'wb').write(r.content)
    print("  Downloaded")

    tarf = tarfile.open(f'{dir_name}.tar.gz', 'r')
    for f in tarf.getnames():
        if not (f.startswith('/') or f.startswith('.')):
            tarf.extract(f)
    tarf.close()
    os.remove(f'{dir_name}.tar.gz')

    print(f"Downloaded and unpacked cTFIDF-{version}.")

import sys
if sys.path[-1] != dir_path:
    sys.path.append(dir_path)

try:
    from ctfidf import CTFIDFVectorizer
    print("Loaded Class-TF/IDF Vectorizer.")
except ModuleNotFoundError:
    print("Still can't load Class-TF/IDF Vectorizer.")

    print("="*25)
    print("You should try restarting the kernel now.\nFor some reason unpacking and loading\nimmediately doesn't work.")
    print("="*25)

Loaded Class-TF/IDF Vectorizer.

 Stop: if you still have errors after running the above code block for the first time then you will probably have to Restart the Kernel at this point. This code is trying to download a new module for which no installer exists and then register it with Python, but the process doesn't seem bullet-proof in my testing. Sorry, but you should only need to restart the Kernel this first time.

The code below tries to find a narrow sans-serif TTF font by path that is slightly nicer than the default for the WordCloud library. You would need to update this default for your own system. You can list available fonts using (/ht imsc):

In [ ]:
import matplotlib.font_manager
fonts = matplotlib.font_manager.findSystemFonts(fontpaths=None, fontext='ttf')
print("Possible fonts: ", end="")
for f in sorted(fonts):
    if 'Narrow' in f:
        print(f.split(os.path.sep)[-1], end=", ")
    elif 'Sans' in f:
        print(f.split(os.path.sep)[-1], end=", ")
print()

In [ ]:
fp = fonts[0] # Ensure at least _something_ is set here
for f in fonts:
    if 'LiberationSansNarrow-Regular' in f:
        fp = f.split(os.path.sep)[-1].split('.')[0]
        break
    elif 'Arial Narrow.ttf' in f:
        fp = f.split(os.path.sep)[-1].split('.')[0]
        break
    elif 'Narrow' in f:
        fp = f.split(os.path.sep)[-1].split('.')[0]
print(f"Using font: {fp}")

fname = ''.join([f' {x}' if x==x.upper() else x for x in fp.split('-')[0]]).strip().replace('  ','')
print(f"  Guessing at font name: {fname}")

# These are font dictionaries for the 's'uper-title, 't'itle,
# 'a'xis, and 'l'abels.
sfont = {'fontname':fname, 'fontsize':16}
tfont = {'fontname':fname, 'fontsize':12}
afont = {'fontname':fname, 'fontsize':10}
lfont = {'fontname':fname, 'fontsize':8}

In [ ]:
# Random seed
rs = 43

# Which embeddings to use
src_embeddings = 'doc_vec'

Load the Data
In this tutorial I make use of the modern Parquet format: it's highly-compressed and columnar, so it works very well (and quickly) with large data sets. The file can also be read directly in DuckDB if you use it, but the general idea is to minimise the volume of data transfered. The columnar orientation means that you can quickly load only the columns that you need for an analysis, and don't have to read in the entire data set each time (as you would with, say, CSV or most other common data formats).